<a href="https://colab.research.google.com/github/alejovillores/tp2_spark_villores_alejo/blob/main/ejercicio_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejercicio 29
Para los contenidos visibles en wikipedia, cuales son los artículos que tienen la
máxima y mínima distancia entre ids de su revisión actual y la anterior


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 198 kB 53.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c321e53724fed51b6e91e2723044130bfe41aaf52d1306652427b7143c6d58b2
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hi

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd


In [ ]:
!pip install pyarrow
contents = pd.read_parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/contents.csv')

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Dice contents.csv pero esta en formato parquet

In [ ]:
sqlContext = SQLContext(sc)
df_contenidos = sqlContext.read.parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/contents.csv', header=True, inferSchema=True)
rdd_contenidos = df_contenidos.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd_contenidos.count()

4132164

In [ ]:
rdd_contenidos.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

Elimino aquellas filas que no poseen un contenido, un revisor parent y un revisor id

In [ ]:
rdd_contenidos_cleaned = rdd_contenidos.filter(lambda x: x.id != None)

In [ ]:
rdd_contenidos_cleaned.count()

4132164

In [ ]:
rdd_contenidos_cleaned.take(4)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [ ]:
help(rdd_contenidos_cleaned)

El `revision_id` es el actual lo cual es mas grande que el `parent_revision` por lo que con su diferencia determino los mas lejos y los mas cercas.

In [ ]:
rdd_nuevo = rdd_contenidos_cleaned.map(lambda x: (x.id,x.revision_id,x.parent_revision))

In [ ]:
rdd_nuevo.take(5)

[(5, 132533307, 132516631.0),
 (7, 138035057, 138027475.0),
 (10, 137966826, 137825165.0),
 (15, 137986788, 133666383.0),
 (17, 130869554, 129413188.0)]

In [ ]:
import math

In [ ]:
def clean_revisors(x):
    parent = x[1]
    rev_id = x[2]
    if parent == None:
      parent = 0
    if rev_id == None:
      rev_id = 0
    
    return (x[0], parent, rev_id)

In [ ]:
grouped_id_distances = rdd_nuevo.map(clean_revisors)

In [ ]:
grouped_id_distances.take(4)

[(5, 132533307, 132516631.0),
 (7, 138035057, 138027475.0),
 (10, 137966826, 137825165.0),
 (15, 137986788, 133666383.0)]

In [ ]:
id_distances = grouped_id_distances.map(lambda x: (int(x[0]),math.fabs(int(x[1]) - int(x[2])) ))

In [ ]:
id_distances.take(20)

[(5, 16676.0),
 (7, 7582.0),
 (10, 141661.0),
 (15, 4320405.0),
 (17, 1456366.0),
 (18, 6901228.0),
 (19, 1.0),
 (20, 113592.0),
 (22, 13799538.0),
 (23, 64.0),
 (24, 2.0),
 (26, 8184181.0),
 (27, 721120.0),
 (28, 4461.0),
 (32, 1.0),
 (33, 101.0),
 (35, 33771.0),
 (36, 4.0),
 (37, 36.0),
 (38, 279761.0)]

Busco el max


In [ ]:
id_distances.reduce(lambda x,y: x if x[1] > y[1] else y) 

(10010598, 138051961.0)

Busco el minimo

In [ ]:
id_distances.reduce(lambda x,y: x if x[1] < y[1] else y) 

(10010546, 1.0)